## Phase II
### Data Cleaning

#### Name: Everett Meehan

In [1]:
import requests
import pandas as pd

api_key = "GodjGTNmZedxuPZHaUdqYCBnTjy5Skh8lG7hrOYO"  # Or your key

# Search Foods
url = "https://api.nal.usda.gov/fdc/v1/foods/search"
params = {"query": "superfood", "api_key": api_key}
response = requests.get(url, params=params)
print(response.json())


{'totalHits': 1107, 'currentPage': 1, 'totalPages': 23, 'pageList': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'foodSearchCriteria': {'query': 'superfood', 'generalSearchInput': 'superfood', 'pageNumber': 1, 'numberOfResultsPerPage': 50, 'pageSize': 50, 'requireAllWords': False}, 'foods': [{'fdcId': 168208, 'description': 'Fruit juice smoothie, ODWALLA, ORIGINAL SUPERFOOD', 'commonNames': '', 'additionalDescriptions': '', 'dataType': 'SR Legacy', 'ndbNumber': 9513, 'publishedDate': '2019-04-01', 'foodCategory': 'Fruits and Fruit Juices', 'allHighlightFields': '', 'score': 321.29324, 'microbes': [], 'foodNutrients': [{'nutrientId': 1187, 'nutrientName': 'Folate, food', 'nutrientNumber': '432', 'unitName': 'UG', 'derivationCode': 'A', 'derivationDescription': 'Analytical', 'derivationId': 1, 'value': 14.0, 'foodNutrientSourceId': 1, 'foodNutrientSourceCode': '1', 'foodNutrientSourceDescription': 'Analytical or derived from analytical', 'rank': 7100, 'indentLevel': 1, 'foodNutrientId': 1338174, 'da

In [20]:
def combine_pages(url, query, key):
  """
  Combine all the pages of data into one dataframe

  Args:
  - url (str): website url
  - query (str): Search Query
  - key (str): Api key

  Returns:
  - df (DataFrame): DataFrame with every page of data

  """
  params = {"query": query,"pageSize":200, "api_key": key}
  response = requests.get(url, params=params)
  pages = response.json()['totalPages']

  df = pd.DataFrame()
  for i in range(pages):
    if i > 25:
      break
    params = {"query": query,"page": i,"pageSize": 200, "api_key": key}
    data = requests.get(url, params=params)
    page = pd.DataFrame(data.json()['foods'])
    df = pd.concat([df,page], axis=0)

  return df

In [21]:
super_df = combine_pages(url, "superfood", api_key)
super_df.shape

(1200, 32)

In [22]:
super_df.head()

,fdcId,description,commonNames,additionalDescriptions,dataType,ndbNumber,publishedDate,foodCategory,allHighlightFields,score,...,marketCountry,modifiedDate,dataSource,packageWeight,servingSizeUnit,servingSize,householdServingFullText,shortDescription,tradeChannels,caffeineStatement
0,168208,"Fruit juice smoothie, ODWALLA, ORIGINAL SUPERFOOD",,,SR Legacy,9513.0,2019-04-01,Fruits and Fruit Juices,,321.293240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2665686,"SUPERFOOD VEGGIE CAKES, SUPERFOOD",NaN,NaN,Branded,NaN,2023-11-16,"Frozen Breakfast Sandwiches, Biscuits & Meals",,72.368450,...,United States,2021-09-07,LI,12 OZ/340 g,GRM,57.0,1 cake,,[NO_TRADE_CHANNEL],NaN
2,2620391,"ORGANIC, SUPERFOODS VEGGIE BURGERS, SUPERFOODS",NaN,NaN,Branded,NaN,2023-08-31,Frozen Patties and Burgers,,48.151413,...,United States,2023-07-14,LI,10 oz/283 g,GRM,71.0,1 PATTY,,[NO_TRADE_CHANNEL],NaN
3,1882647,"SUPERFOOD PESTO SAUCE, SUPERFOOD PESTO",NaN,NaN,Branded,NaN,2021-07-29,Prepared Pasta & Pizza Sauces,,48.151413,...,United States,2018-04-25,LI,7 oz/198 g,g,62.0,0.25 cup,NaN,[NO_TRADE_CHANNEL],NaN
4,2145788,"SUPERFOOD POWER SALAD, SUPERFOOD POWER",NaN,NaN,Branded,NaN,2021-10-28,"Pickles, Olives, Peppers & Relishes",,48.151413,...,United States,2019-10-07,LI,9.25 oz/262 g,g,100.0,1 cup,NaN,[NO_TRADE_CHANNEL],NaN


In [23]:
def add_nutrients(df):
  """
  extracts nutrition information from foodNutrients dictionary
  and creates new columns

  Args:
  - df (DataFrame): all the food data

  Returns:
  - df (DaataFrame): updated dataframe with nutrition columns
  """
  for index, row in df.iterrows():
      if isinstance(row['foodNutrients'], list):
          fat = 0
          protein = 0
          carbs = 0
          for nutrient_info in row['foodNutrients']:
              nutrient_name = nutrient_info.get('nutrientName')
              nutrient_unit = nutrient_info.get('unitName')
              nutrient_value = nutrient_info.get('value')

              if 'Fatty acids' in nutrient_name:
                fat += nutrient_value
              elif nutrient_name == 'Protein':
                  protein = nutrient_value
              elif nutrient_name == 'Carbohydrate, by difference':
                  carbs = nutrient_value


              if nutrient_name and nutrient_unit:
                  column_name = f'{nutrient_name} ({nutrient_unit})'
                  df.at[index, column_name] = nutrient_value
          df.at[index, 'Total Fat (G)'] = fat
          df.at[index, 'Calories'] = (protein * 4) + (carbs * 4) + (fat * 9)
  return df

In [24]:
super_df = add_nutrients(super_df)

/tmp/ipython-input-3139376305.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, column_name] = nutrient_value
/tmp/ipython-input-3139376305.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, column_name] = nutrient_value
/tmp/ipython-input-3139376305.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame

In [25]:
super_df.head()

,fdcId,description,commonNames,additionalDescriptions,dataType,ndbNumber,publishedDate,foodCategory,allHighlightFields,score,...,Vitamin K (phylloquinone) (UG),Total Fat (G),Calories,"Sugars, added (G)",Total sugar alcohols (G),"Vitamin D (D2 + D3), International Units (IU)",Pantothenic acid (MG),Vitamin E (MG_ATE),"Molybdenum, Mo (UG)",Vitamin E (MG)
0,168208,"Fruit juice smoothie, ODWALLA, ORIGINAL SUPERFOOD",,,SR Legacy,9513.0,2019-04-01,Fruits and Fruit Juices,,321.293240,...,5.0,0.261,50.869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2665686,"SUPERFOOD VEGGIE CAKES, SUPERFOOD",NaN,NaN,Branded,NaN,2023-11-16,"Frozen Breakfast Sandwiches, Biscuits & Meals",,72.368450,...,NaN,0.000,70.040,1.8,NaN,NaN,NaN,NaN,NaN,NaN
2,2620391,"ORGANIC, SUPERFOODS VEGGIE BURGERS, SUPERFOODS",NaN,NaN,Branded,NaN,2023-08-31,Frozen Patties and Burgers,,48.151413,...,NaN,1.410,136.810,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1882647,"SUPERFOOD PESTO SAUCE, SUPERFOOD PESTO",NaN,NaN,Branded,NaN,2021-07-29,Prepared Pasta & Pizza Sauces,,48.151413,...,NaN,7.260,123.420,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2145788,"SUPERFOOD POWER SALAD, SUPERFOOD POWER",NaN,NaN,Branded,NaN,2021-10-28,"Pickles, Olives, Peppers & Relishes",,48.151413,...,NaN,0.000,84.000,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
print(super_df.columns.tolist())

['fdcId', 'description', 'commonNames', 'additionalDescriptions', 'dataType', 'ndbNumber', 'publishedDate', 'foodCategory', 'allHighlightFields', 'score', 'microbes', 'foodNutrients', 'finalFoodInputFoods', 'foodMeasures', 'foodAttributes', 'foodAttributeTypes', 'foodVersionIds', 'gtinUpc', 'brandOwner', 'brandName', 'subbrandName', 'ingredients', 'marketCountry', 'modifiedDate', 'dataSource', 'packageWeight', 'servingSizeUnit', 'servingSize', 'householdServingFullText', 'shortDescription', 'tradeChannels', 'caffeineStatement', 'Folate, food (UG)', 'Vitamin B-12, added (UG)', 'Cholesterol (MG)', 'Fatty acids, total trans (G)', 'Fatty acids, total saturated (G)', 'SFA 8:0 (G)', 'SFA 12:0 (G)', 'SFA 14:0 (G)', 'Vitamin A, RAE (UG)', 'Carotene, beta (UG)', 'Carotene, alpha (UG)', 'Vitamin D (D2 + D3) (UG)', 'Cryptoxanthin, beta (UG)', 'Lycopene (UG)', 'Vitamin C, total ascorbic acid (MG)', 'Thiamin (MG)', 'Riboflavin (MG)', 'Folate, total (UG)', 'Vitamin B-12 (UG)', 'Protein (G)', 'Ash (G

In [27]:
def clean(df):
  """
  Cleans out the dataframe

  Args:
  - df (DataFrame): Food dataframe

  Returns:
  - df_cleaned (DataFrame): cleaned dataframe
  """
  macros = ['Protein (G)', 'Carbohydrate, by difference (G)', 'Total Fat (G)', 'Calories']
  df_cleaned = df.dropna(subset = macros)
  micro_nulls_per_row = df_cleaned.iloc[:,32:147].isnull().sum(axis=1)
  df_cleaned['<70% micronutrients completeness'] = (micro_nulls_per_row / 114) < 0.7

  return df_cleaned

In [28]:
super_cleaned = clean(super_df)
super_cleaned.shape

/tmp/ipython-input-2380346602.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cleaned['<70% micronutrients completeness'] = (micro_nulls_per_row / 114) < 0.7
/tmp/ipython-input-2380346602.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['<70% micronutrients completeness'] = (micro_nulls_per_row / 114) < 0.7


(1176, 148)

In [29]:
super_cleaned.to_csv('superfoods_tagged.csv', index=False)

In [13]:
api_key = "GodjGTNmZedxuPZHaUdqYCBnTjy5Skh8lG7hrOYO"  # Or your key

# Search Foods
url = "https://api.nal.usda.gov/fdc/v1/foods/search"
params = {"query": "food","pagesize":200, "api_key": api_key}
response = requests.get(url, params=params)
print(response.json())

{'totalHits': 114343, 'currentPage': 1, 'totalPages': 2287, 'pageList': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'foodSearchCriteria': {'query': 'food', 'generalSearchInput': 'food', 'pageNumber': 1, 'numberOfResultsPerPage': 50, 'pageSize': 50, 'requireAllWords': False}, 'foods': [{'fdcId': 588394, 'description': 'FOOD', 'dataType': 'Branded', 'gtinUpc': '8997210211017', 'publishedDate': '2019-04-01', 'brandOwner': 'GOLDEN SEAHORSE', 'ingredients': 'SHRIMP, SALT AND SODIUM TRIPOLYPHOSPHATE (FOR MOISTURE RETENTION)', 'marketCountry': 'United States', 'foodCategory': 'Frozen Fish & Seafood', 'modifiedDate': '2018-07-06', 'dataSource': 'LI', 'servingSizeUnit': 'g', 'servingSize': 85.0, 'householdServingFullText': '3 ONZ', 'tradeChannels': ['NO_TRADE_CHANNEL'], 'allHighlightFields': '', 'score': 1221.0885, 'microbes': [], 'foodNutrients': [{'nutrientId': 1005, 'nutrientName': 'Carbohydrate, by difference', 'nutrientNumber': '205', 'unitName': 'G', 'derivationCode': 'LCCD', 'derivationDescription'

In [30]:
other_df = combine_pages(url, "food", api_key)
other_df.head()

,fdcId,description,dataType,gtinUpc,publishedDate,brandOwner,ingredients,marketCountry,foodCategory,modifiedDate,...,foodAttributes,foodAttributeTypes,foodVersionIds,scientificName,commonNames,additionalDescriptions,ndbNumber,tags,foodCode,foodCategoryId
0,588394,FOOD,Branded,8997210211017,2019-04-01,GOLDEN SEAHORSE,"SHRIMP, SALT AND SODIUM TRIPOLYPHOSPHATE (FOR ...",United States,Frozen Fish & Seafood,2018-07-06,...,[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,169705,Oats (Includes foods for USDA's Food Distribut...,SR Legacy,NaN,2019-04-01,NaN,NaN,NaN,Cereal Grains and Pasta,NaN,...,[],[],[],Avena sativa L.,,,20038.0,NaN,NaN,NaN
2,2644260,The Development of the Davis Food Glycopedia—A...,Experimental,NaN,2023-10-26,NaN,NaN,NaN,NaN,NaN,...,[],[],[],NaN,NaN,NaN,NaN,"[Database, USDA Research]",NaN,NaN
3,170302,"Fast food, biscuit",SR Legacy,NaN,2019-04-01,NaN,NaN,NaN,Fast Foods,NaN,...,[],[],[],NaN,,,21142.0,NaN,NaN,NaN
4,170300,"Fast foods, coleslaw",SR Legacy,NaN,2019-04-01,NaN,NaN,NaN,Fast Foods,NaN,...,[],[],[],NaN,,,21127.0,NaN,NaN,NaN


In [31]:
other_df.shape

(5200, 31)

In [32]:
other_df = add_nutrients(other_df)
other_df.head()

/tmp/ipython-input-3139376305.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, column_name] = nutrient_value
/tmp/ipython-input-3139376305.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, column_name] = nutrient_value
/tmp/ipython-input-3139376305.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame

,fdcId,description,dataType,gtinUpc,publishedDate,brandOwner,ingredients,marketCountry,foodCategory,modifiedDate,...,PUFA 20:4 n-6 (G),PUFA 21:5 (G),Hydroxyproline (G),SFA 13:0 (G),Vitamin D2 (ergocalciferol) (UG),"TFA 18:2 t,t (G)",Campesterol (MG),Stigmasterol (MG),Beta-sitosterol (MG),PUFA 18:2 i (G)
0,588394,FOOD,Branded,8997210211017,2019-04-01,GOLDEN SEAHORSE,"SHRIMP, SALT AND SODIUM TRIPOLYPHOSPHATE (FOR ...",United States,Frozen Fish & Seafood,2018-07-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,169705,Oats (Includes foods for USDA's Food Distribut...,SR Legacy,NaN,2019-04-01,NaN,NaN,NaN,Cereal Grains and Pasta,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2644260,The Development of the Davis Food Glycopedia—A...,Experimental,NaN,2023-10-26,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,170302,"Fast food, biscuit",SR Legacy,NaN,2019-04-01,NaN,NaN,NaN,Fast Foods,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,170300,"Fast foods, coleslaw",SR Legacy,NaN,2019-04-01,NaN,NaN,NaN,Fast Foods,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
other_cleaned = clean(other_df)
other_cleaned.shape

/tmp/ipython-input-2380346602.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cleaned['<70% micronutrients completeness'] = (micro_nulls_per_row / 114) < 0.7
/tmp/ipython-input-2380346602.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['<70% micronutrients completeness'] = (micro_nulls_per_row / 114) < 0.7


(5122, 182)

In [34]:
other_cleaned.to_csv("Foods_tagged", index=False)